In [1]:
import requests
import pandas as pd
import io
from collections import Counter



class Dataset:

    def __init__(self):
        self.run_all()

    def run_all(self):
        self.original_dataset = self.get_data()
        self.attributes = self.get_attributes_names(self.original_dataset) #2.1.1
        print(f"{len(self.attributes)} atributos descrevem o dataset original\n")
        self.confirmed_cases = self.get_confirmed_cases(self.original_dataset) #2.1.2
        self.min_date, self.max_date = self.min_max_dates(self.confirmed_cases) #2.1.3
        
        # Limpeza e organização do dataset
        self.cleaned_data, self.after_clean_number = self.clean_data(self.original_dataset)
        self.after_clean_attributes = self.get_attributes_names(self.cleaned_data)
        print(f"{len(self.after_clean_attributes)} atributos descrevem o dataset após a limpeza\n")
        
        # Porcentagem de recuperados
        self.confirmed_cases_after_clean = self.get_confirmed_cases(self.cleaned_data)
        keys, rec_values, self.rec_percentage = self.recovered_percentage(self.confirmed_cases_after_clean)
        
        # Sexo mais afetado
        keys, values, self.sex_most_infected = self.m_and_f_confirmed_percentage(self.confirmed_cases_after_clean) 

        # Media e desvio padrao
        desvio_padrao, media, menor, maior = self.media_desviopadrao_idade(self.confirmed_cases_after_clean)
        
        # Bairro mais afetado
        most_affected_neighbor = self.most_affected_neighborhood(self.confirmed_cases_after_clean)
        
        # Três bairros com mais recuperados
        three_neighbors = self.three_neighbors_most_recovered_cases(self.confirmed_cases_after_clean)
        
        # Tipos de testes efetuados
        nome_testes, qtde_testes, perc_testes = self.testes(self.cleaned_data)
        
        # Taxa de letalidade
        taxa_let = self.letalidade(self.confirmed_cases_after_clean)
        
        # Correlação
        correl = self.correlacao(self.confirmed_cases_after_clean)
        
        pass
        
    def get_updated_data(self):
        request = requests.get('https://covid19.manaus.am.gov.br/wp-content/uploads/Manaus.csv', verify=False)
        content=request.content
        dataset=pd.read_csv(io.StringIO(content.decode('latin-1')), error_bad_lines=False, sep=';')
        return dataset

    def get_data(self):
        data = pd.read_csv('Manaus.csv', sep=';', encoding='latin-1')
        return data
    
    def get_confirmed_cases(self, data):
        confirmed_cases = data.loc[data['_classificacao'] == 'Confirmado']
        print(f"Foram confirmados {confirmed_cases.shape[0]} casos\n")
        return confirmed_cases

    def get_cases_infos(self, data):
        possible_values = list(data['_classificacao'])
        keys = Counter(possible_values).keys()
        values = Counter(possible_values).values()
        return keys, values

    def get_attributes_names(self, data):
        original_attributes = data.columns.tolist()
        removed_underline = [i[1:] for i in original_attributes]
        print(f"Os atributos presentes no dataset são: {removed_underline}\n")
        return removed_underline

    def min_max_dates(self, data):
        # the date is given by dt_notificacao attribute
        dates = pd.to_datetime(data['_dt_notificacao'], format='%d/%m/%Y')
        min_date = dates.min()
        max_date = dates.max()
        print(f"O caso mais antigo foi em {min_date} e o mais recente em {max_date}\n")
        return min_date, max_date

    def clean_data(self, data):
        cleaned_data = data[['_idade','_sexo','_bairro','_classificacao','_conclusao','_dt_notificacao','_teste_pcr','_teste_anticorpo','_teste_antigeno','_teste_igm','_teste_igg','_faixa etária']]
        initial_number = cleaned_data.shape[0]
        print(f'Numero de exemplos antes da limpeza: {initial_number}\n')
        cleaned_data = cleaned_data.dropna()
        cleaned_data = cleaned_data.reset_index(drop=True)
        after_clean_number = cleaned_data.shape[0]
        print(f'Numero de linhas após a limpeza: {after_clean_number}\n')
        cleaned_data = cleaned_data.reset_index(drop=True)
        return cleaned_data, after_clean_number

    def recovered_percentage(self, data):
        # Porcentagem de individuos recuperados em relação a todos os casos confirmados
        possible_values = list(data['_conclusao'])
        keys = Counter(possible_values).keys()
        values = Counter(possible_values).values()
        values_list = list(values)
        percentage = (values_list[0]/(sum(values_list)))*100 
        print(f'Porcentagem de individuos recuperados: {percentage:.2f}%\n')
        return (keys, values, percentage)

    def m_and_f_confirmed_percentage(self, data):
        possible_values = list(data['_sexo'])
        keys = Counter(possible_values).keys()
        values = Counter(possible_values).values()
        m_f_keys = list(keys)
        m_f_values = list(values)
        bigger_values = max(m_f_values)
        index = m_f_values.index(bigger_values)
        most_affected = m_f_keys[index]
        print(f'Os casos acometeram mais: {most_affected}\n')
        return m_f_keys, m_f_values, most_affected

    def most_affected_neighborhood(self, data):
        possible_values = list(data['_bairro'])
        keys = Counter(possible_values).keys()
        values = Counter(possible_values).values()
        neighbor_keys = list(keys)
        neighbor_values = list(values)
        bigger_values = max(neighbor_values)
        index = neighbor_values.index(bigger_values)
        most_affected = neighbor_keys[index]
        print(f'O bairro mais afetado foi: {most_affected}\n')
        return most_affected

    def three_neighbors_most_recovered_cases(self, data):
        possible_values = data[['_bairro', '_conclusao']]
        recovered_values = possible_values.loc[possible_values['_conclusao'] == 'Recuperado']
        recovered_neighbors = recovered_values['_bairro']
        keys = Counter(recovered_neighbors).keys()
        values = Counter(recovered_neighbors).values()
        neighbor_keys = list(keys)
        neighbor_values = list(values)
        sorted_neighbors = sorted(list(zip(neighbor_values, neighbor_keys)), reverse=True)[:3]
        print(f'Os 3 maiores: {sorted_neighbors}\n')
        return sorted_neighbors

    def media_desviopadrao_idade(self, data):
        possible_values = data[['_idade', '_classificacao']]
        
        idades = possible_values['_idade']

        desvio_padrao = possible_values['_idade'].std()
        media = possible_values['_idade'].mean()
        menor = possible_values['_idade'].min()
        maior = possible_values['_idade'].max()

        print(f"O desvio padrão das idade é: {desvio_padrao}\n")
        print(f"A media das idades é: {media}\n")
        print(f"O mais jovem tem: {menor} anos\n")
        print(f"O mais velho tem: {maior} anos\n")
        
        return desvio_padrao, media, menor, maior

    def testes(self, data):
        # Quantidade de cada tipo de teste
        total_teste_pcr = data['_teste_pcr'].sum()
        total_teste_anticorpo = data['_teste_anticorpo'].sum()
        total_teste_antigeno = data['_teste_antigeno'].sum()
        total_teste_igm = data['_teste_igm'].sum()
        total_teste_igg = data['_teste_igg'].sum()

        nome_testes = ['_teste_pcr', '_teste_anticorpo', '_teste_antigeno', '_teste_igm', '_teste_igg']

        qtde_testes = [total_teste_pcr, total_teste_anticorpo, total_teste_antigeno, total_teste_igm, total_teste_igg]

        # Total de testes realizados
        total_testes_realizados = total_teste_pcr + total_teste_anticorpo + total_teste_antigeno + total_teste_igm + total_teste_igg

        # Porcentagem de cada tipo de teste com relacao ao total de testes realizados
        perc_teste_pcr = (total_teste_pcr/total_testes_realizados) * 100
        perc_teste_anticorpo = (total_teste_anticorpo/total_testes_realizados) * 100
        perc_teste_antigeno = (total_teste_antigeno/total_testes_realizados) * 100
        perc_teste_igm = (total_teste_igm/total_testes_realizados) * 100
        perc_teste_igg = (total_teste_igg/total_testes_realizados) * 100

        perc_testes = [perc_teste_pcr, perc_teste_anticorpo, perc_teste_antigeno, perc_teste_igm, perc_teste_igg]
        print(f"Os teste efetuados foram: {nome_testes}\n")
        print(f"O número de testes efetuado, respectivamente, é: {qtde_testes}\n")
        print(f"A porcentagem dos tipos de teste efetuados, respectivamente, são: {perc_testes}\n")
        
        return nome_testes, qtde_testes, perc_testes

    def letalidade(self, data):
        confirmed_cases = data.loc[data['_classificacao'] == 'Confirmado']
        total_mortos = data.loc[data['_conclusao'] == 'Óbito']
        taxa = (total_mortos.shape[0]/confirmed_cases.shape[0])*100
        print(f"A taxa de letalidade calculada é: {taxa}\n")
        return taxa

    def correlacao(self, data):
        possible_values = data[['_idade', '_classificacao']]
        confirmed_values = possible_values.loc[possible_values['_classificacao'] == 'Confirmado']

        idades = confirmed_values.groupby('_idade')['_classificacao'].value_counts().reset_index(name='_qtde')

        correl = idades['_idade'].corr(idades['_qtde'])     
        print(f"A correlação calculada entre a idade e o número de casos é: {correl}\n")
        
        return correl

data = Dataset()

/home/rafa/meu_projeto/meu_projeto/lib/python3.6/site-packages/ipykernel_launcher.py:208: DtypeWarning: Columns (15,21,23,29,30,31,32,34) have mixed types.Specify dtype option on import or set low_memory=False.


Os atributos presentes no dataset são: ['idade', 'faixa etária', 'sexo', 'bairro', 'classificacao', 'comorb_renal', 'comorb_diabetes', 'comorb_imuno', 'comorb_cardio', 'conclusao', 'dt_notificacao', 'taxa', 'dt_evolucao', 'raca', 'dt_sintomas', 'criterio', 'sintoma_garganta', 'sintoma_dispneia', 'sintoma_febre', 'sintoma_tosse', 'sintoma_outros', 'etnia', 'profiss_saude', 'srag', 'se_notificacao', 'distrito', 'bairro_mapa', 'comorb_respiratoria', 'comorb_cromossomica', 'comorb_hepatica', 'comorb_neurologica', 'comorb_hemato', 'comorb_obessidade', 'origem', 'evolução', 'teste_pcr', 'teste_anticorpo', 'teste_antigeno', 'teste_igm', 'teste_igg']

40 atributos descrevem o dataset original

Foram confirmados 38347 casos

O caso mais antigo foi em 2020-02-16 00:00:00 e o mais recente em 2020-08-12 00:00:00

Numero de exemplos antes da limpeza: 113534

Numero de linhas após a limpeza: 54659

Os atributos presentes no dataset são: ['idade', 'sexo', 'bairro', 'classificacao', 'conclusao', 'dt_n